In [37]:
!pip install pymongo


In [38]:
import json
from pymongo import MongoClient

# Connexion à la base de données MongoDB
client = MongoClient('localhost', 27017)
database = client["rhobs"]
collection = database["people"]

# Chargement des données JSON et insertion dans la collection
with open("data.json.codechallenge.janv22.RHOBS.json", "r") as fp:
    data = json.load(fp)
    for person in data:
        collection.insert_one(person)

In [39]:
nb_femmes = collection.count_documents({'sex': 'F'})
nb_hommes = collection.count_documents({'sex': 'M'})

print(f"Nombre de femmes : {nb_femmes}")
print(f"Nombre d'hommes : {nb_hommes}")

Nombre de femmes : 350588
Nombre d'hommes : 349412


In [45]:
def entreprise_des_personnes_plus_de_Nemployer(N):
    pipeline = [
        {"$group": {"_id": "$company", "count": {"$sum": 1}}},
        {"$match": {"count": {"$gt": N}}},
        {"$sort": {"count": -1}}
    ]
    result = list(collection.aggregate(pipeline))
    companies = [r['_id'] for r in result]
    return companies


In [104]:
companies = entreprise_des_personnes_plus_de_Nemployer(1000)
print(companies)


['Gay', 'Evrard', 'Bonnet', 'Normand', 'Leclercq', 'Pottier', 'Bernard', 'Monnier', 'Rousset', 'Colin', 'Durand', 'Roussel', 'Martins', 'Lecomte', 'Martinez', 'Pages', 'Coste', 'Rodriguez', 'Camus', 'Guilbert', 'Dijoux', 'Albert', 'Colas', 'Guichard', 'Gros', 'Bailly', 'Duhamel', 'Launay', 'Blanc', 'Laurent', 'Faure', 'Leduc', 'Charrier', 'Ollivier', 'Laine', 'Hubert', 'Gilbert', 'Valentin', 'Raymond', 'Hervé', 'Jacques', 'Lombard', 'Wagner', 'Marques', 'Perez', 'Garnier', 'Renaud', 'Hamon', 'Clerc', 'Imbert', 'Rolland', 'Bourgeois', 'Legendre', 'Bourdon', 'Gaillard', 'Vallée']


In [113]:
from datetime import datetime

def pyramid_de_age_pour_métier(job):
    pipeline = [
        {"$set": {"birthdate": {"$toDate": "$birthdate"}}},
        {"$match": {"job": job}},
        {"$group": {
            "_id": {
                "lower": {"$trunc": {"$divide": [{"$subtract": [datetime.now(), "$birthdate"]}, 31536000000]}},
                "upper": {"$subtract": [
                    {"$trunc": {"$divide": [{"$subtract": [datetime.now(), "$birthdate"]}, 31536000000]}},
                    {"$mod": [{"$trunc": {"$divide": [{"$subtract": [datetime.now(), "$birthdate"]}, 31536000000]}}, 5]}
                ]}
            },
            "count": {"$sum": 1}
        }},
        {"$sort": {"_id.lower": 1}}
    ]
    
    result = list(collection.aggregate(pipeline))
    
    age_pyramid = {}
    for item in result:
        age_group = f"{item['_id']['lower']}-{item['_id']['upper']}"
        age_pyramid[age_group] = age_pyramid.get(age_group, {"M": 0, "F": 0})
        age_pyramid[age_group]["M" if item["_id"]["lower"] % 10 == 0 else "F"] += item["count"]
    
    return age_pyramid
print(pyramid_de_age_pour_métier("éducateur sportif sportive des activités aquatiques et de la natation"))

{'2.0-0.0': {'M': 0, 'F': 14}, '4.0-0.0': {'M': 0, 'F': 28}, '5.0-5.0': {'M': 0, 'F': 14}, '6.0-5.0': {'M': 0, 'F': 14}, '7.0-5.0': {'M': 0, 'F': 28}, '9.0-5.0': {'M': 0, 'F': 14}, '10.0-10.0': {'M': 14, 'F': 0}, '13.0-10.0': {'M': 0, 'F': 14}, '17.0-15.0': {'M': 0, 'F': 14}, '18.0-15.0': {'M': 0, 'F': 14}, '20.0-20.0': {'M': 14, 'F': 0}, '22.0-20.0': {'M': 0, 'F': 28}, '23.0-20.0': {'M': 0, 'F': 14}, '25.0-25.0': {'M': 0, 'F': 14}, '27.0-25.0': {'M': 0, 'F': 14}, '29.0-25.0': {'M': 0, 'F': 14}, '30.0-30.0': {'M': 14, 'F': 0}, '31.0-30.0': {'M': 0, 'F': 14}, '32.0-30.0': {'M': 0, 'F': 14}, '34.0-30.0': {'M': 0, 'F': 14}, '36.0-35.0': {'M': 0, 'F': 28}, '37.0-35.0': {'M': 0, 'F': 28}, '41.0-40.0': {'M': 0, 'F': 14}, '42.0-40.0': {'M': 0, 'F': 14}, '50.0-50.0': {'M': 14, 'F': 0}, '52.0-50.0': {'M': 0, 'F': 14}, '56.0-55.0': {'M': 0, 'F': 28}, '59.0-55.0': {'M': 0, 'F': 14}, '61.0-60.0': {'M': 0, 'F': 28}, '63.0-60.0': {'M': 0, 'F': 14}, '67.0-65.0': {'M': 0, 'F': 14}, '71.0-70.0': {'M': 